<a href="https://colab.research.google.com/github/ylepen/NPL--M2-DG/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt



# Step 0: Importing Data

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pathlib
data_sentiment_dir = "gdrive/My Drive/data sentiment analysis"
data_sentiment_dir = pathlib.Path(data_sentiment_dir)

In [5]:
df = pd.read_json(data_sentiment_dir / "reviews_.json",lines=True)

In [ ]:
df.sample(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
165777,AHAOXE7IWCFPA,B003HM4ZD6,Sasha W,"[0, 0]",This item arrived relatively fast and came jus...,5,Blackberry Housing,1316044800,"09 15, 2011"
158799,A5WPW331QUGTB,B003CIYMI8,pleased,"[0, 0]",Gave as a present and he was thrilled with it....,5,hockey fan,1342915200,"07 22, 2012"
124242,A1CVRDQVG1H383,B002OG45CM,JOHN,"[0, 0]",IT WORKS AS STATED. IT GIVES A SILKSCREEN TEXT...,5,A MUST HAVE.,1375056000,"07 29, 2013"
549471,A2OAZR45DFR8Q6,B00H4ER2TM,Michael Wilson,"[1, 1]",Very good quality coin. It's got a good feel t...,5,Awesome coin for an amazing price.,1404172800,"07 1, 2014"
194916,AG7TJVU9CZFZZ,B00439M5L6,"Nancy A. Treder ""4tograph4""","[0, 0]",Fits perfectly! Pina Coladas are easy with th...,5,"Have blender, will make pina coladas.",1385856000,"12 1, 2013"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 563497 entries, 0 to 563496
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   reviewerID      563497 non-null  object
 1   asin            563497 non-null  object
 2   reviewerName    561709 non-null  object
 3   helpful         563497 non-null  object
 4   reviewText      563497 non-null  object
 5   overall         563497 non-null  int64 
 6   summary         563497 non-null  object
 7   unixReviewTime  563497 non-null  int64 
 8   reviewTime      563497 non-null  object
dtypes: int64(2), object(7)
memory usage: 38.7+ MB


In [6]:
df.rename(columns={"reviewText":"text"},inplace=True)
df = df[df['text'].apply(len)!=0]

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
from nltk.tokenize import sent_tokenize
df['number_of_sentences']=df['text'].apply(lambda text : len(sent_tokenize(text)))

In [9]:
df=df[df['number_of_sentences']==1]
df["text_orig"]=df['text'].copy()

<ipython-input-9-a2230a95248d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text_orig"]=df['text'].copy()


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 75184 entries, 33 to 563494
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   reviewerID           75184 non-null  object
 1   asin                 75184 non-null  object
 2   reviewerName         74451 non-null  object
 3   helpful              75184 non-null  object
 4   text                 75184 non-null  object
 5   overall              75184 non-null  int64 
 6   summary              75184 non-null  object
 7   unixReviewTime       75184 non-null  int64 
 8   reviewTime           75184 non-null  object
 9   number_of_sentences  75184 non-null  int64 
 10  text_orig            75184 non-null  object
dtypes: int64(3), object(8)
memory usage: 6.9+ MB


In [11]:
# Assigning a label based on the overall product rating
df['sentiment']=0
df.loc[df['overall']>=3,'sentiment']=1
df.loc[df['overall']<3,'sentiment']=0

In [12]:
# Removing unnecessary columns
df.drop(columns=['reviewTime','unixReviewTime','overall','reviewerID','reviewerName','summary'],inplace=True)
df.sample(3)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 75184 entries, 33 to 563494
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   asin                 75184 non-null  object
 1   helpful              75184 non-null  object
 2   text                 75184 non-null  object
 3   number_of_sentences  75184 non-null  int64 
 4   text_orig            75184 non-null  object
 5   sentiment            75184 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 4.0+ MB


Reducing the size of the data set (for the illustration as the speed of training the model is too low with all data)

In [15]:
df=df.groupby('sentiment', group_keys=False).apply(lambda x: x.sample(frac=0.02))

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1504 entries, 462933 to 313858
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   asin                 1504 non-null   object
 1   helpful              1504 non-null   object
 2   text                 1504 non-null   object
 3   number_of_sentences  1504 non-null   int64 
 4   text_orig            1504 non-null   object
 5   sentiment            1504 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 82.2+ KB


# **Step I: Loading Model and Tokenization**

In [17]:
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification

config=BertConfig.from_pretrained('bert-base-uncased', finetuning_tasks='binary')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
def get_tokens(text, tokenizer,max_seq_length, add_special_tokens=True):
  input_ids=tokenizer.encode(text,
                             add_special_tokens=add_special_tokens,
                             max_length=max_seq_length,
                             pad_to_max_length=True)
  attention_mask=[int(id>0) for id in input_ids]
  assert len(input_ids) == max_seq_length
  assert len(attention_mask) == max_seq_length
  return (input_ids,attention_mask)

In [19]:
text = "here is the sentence I want embeddings for."
input_ids, attention_mask = get_tokens(text,
                                       tokenizer,
                                       max_seq_length=30,
                                       add_special_tokens=True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
input_tokens=tokenizer.convert_ids_to_tokens(input_ids)
print(text)
print(input_tokens)
print(input_ids)
print(attention_mask)

here is the sentence I want embeddings for.
['[CLS]', 'here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[101, 2182, 2003, 1996, 6251, 1045, 2215, 7861, 8270, 4667, 2015, 2005, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test,Y_train,Y_test = train_test_split(df['text_orig'],
                                                  df['sentiment'],
                                                  test_size=0.2,
                                                  random_state=42,
                                                  stratify=df['sentiment'])

In [21]:
X_train_tokens=X_train.apply(get_tokens, args=(tokenizer,50))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [22]:
X_test_tokens=X_test.apply(get_tokens, args=(tokenizer,50))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [23]:
import torch
from torch.utils.data import TensorDataset

In [24]:
input_ids_train = torch.tensor([features[0] for features in X_train_tokens.values], dtype=torch.long)

In [36]:
input_ids_test = torch.tensor([features[0] for features in X_test_tokens.values], dtype=torch.long)

In [25]:
input_mask_train = torch.tensor([features[1] for features in X_train_tokens.values], dtype=torch.long)

In [37]:
input_mask_test = torch.tensor([features[1] for features in X_test_tokens.values], dtype=torch.long)

In [26]:
label_ids_train = torch.tensor(Y_train.values, dtype=torch.long)

In [38]:
label_ids_test = torch.tensor(Y_test.values, dtype=torch.long)

In [27]:
print(input_ids_train.shape)
print(input_mask_train.shape)
print(label_ids_train.shape)

torch.Size([1203, 50])
torch.Size([1203, 50])
torch.Size([1203])


# Model training

In [28]:
train_dataset = TensorDataset(input_ids_train, input_mask_train,label_ids_train)

In [39]:
test_dataset = TensorDataset(input_ids_test, input_mask_test,label_ids_test)

In [29]:
from torch.utils.data import DataLoader, RandomSampler
train_batch_size = 64
num_train_epoch=2
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset,
                              sampler=train_sampler,
                              batch_size=train_batch_size)
t_total = len(train_dataloader)//num_train_epoch
print("Num examples =", len(train_dataset))


Num examples = 1203


In [30]:
print("Num epoch = ", num_train_epoch)
print("Total train batch size", num_train_epoch)
print("Total optimization steps", t_total)

Num epoch =  2
Total train batch size 2
Total optimization steps 9


In [31]:
from transformers import AdamW, get_linear_schedule_with_warmup
learning_rate = 1e-4
adam_epsilon = 1e-8
warmup_steps =0

In [32]:
optimizer=torch.optim.AdamW(model.parameters(),lr=learning_rate,eps=adam_epsilon)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=warmup_steps,
                                            num_training_steps=t_total)

In [33]:
import tqdm
from tqdm import trange, notebook
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_iterator = trange(num_train_epoch,desc="Epoch")
model.train()



Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [34]:
for epoch in train_iterator:
  epoch_iterator = notebook.tqdm(train_dataloader, desc="Iteration")
  for step, batch in enumerate(epoch_iterator):
    #Reset all gradients at start of every iteration
    model.zero_grad()
    # Put the model and the input observations to GPU
    model.to(device)
    batch = tuple(t.to(device) for t in batch)
    # Indentify the inputs to the model
    inputs={'input_ids': batch[0],
            'attention_mask' : batch[1],
            'labels': batch[2]}
    #Forward Pass through the model. Input -> Model -> Output
    outputs = model(**inputs)
    # Determine the deviation (loss)
    loss = outputs[0]
    print("\r%f" % loss, end='')
    # Back-propagate the loss (automatically calculates gradients)
    loss.backward()
    # Prevent exploding gradients by limiting gradients to 1.0
    torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
    #update the parameters and learning rate
    optimizer.step()
    scheduler.step()

model.save_pretrained('outputs')

Iteration:   0%|          | 0/19 [00:00<?, ?it/s]

0.323494

Epoch:  50%|█████     | 1/2 [10:16<10:16, 616.31s/it]

Iteration:   0%|          | 0/19 [00:00<?, ?it/s]

0.510799

Epoch: 100%|██████████| 2/2 [20:00<00:00, 600.18s/it]


# Step 3 Model Evaluation

In [40]:
from torch.utils.data import SequentialSampler

test_batch_size = 64
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset,
                             sampler=test_sampler,
                             batch_size=test_batch_size)

In [ ]:
#model = model.from_pretrained('/outputs')

In [48]:
# Initialize the prediction and actual labels
preds = None
out_label_ids = None
# Put model in "eval" mode
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [49]:
for batch in notebook.tqdm(test_dataloader,desc="Evaluating"):
  model.to(device)
  batch = tuple(t.to(device) for t in batch)
  with torch.no_grad():
    inputs={'input_ids': batch[0],
            'attention_mask':batch[1],
            'labels':batch[2]}
  outputs = model(**inputs)
  tmp_eval_loss, logits = outputs[:2]
  if preds is None:
    preds = logits.detach().cpu().numpy()
    out_label_ids = inputs['labels'].detach().cpu().numpy()
  else:
    preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
    out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(),axis=0)



Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

In [55]:
from sklearn.metrics import accuracy_score
#preds=np.argmax(preds, axis=1)
acc_score = accuracy_score(preds, label_ids_test)
print('Accuracy Score on Test data', acc_score)

Accuracy Score on Test data 0.8438538205980066
